In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Subil Varghese Jacob\\OneDrive\\Desktop\\Books\\Oil and gas\\Well_Optimization_ML\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Subil Varghese Jacob\\OneDrive\\Desktop\\Books\\Oil and gas\\Well_Optimization_ML'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name : str
    target_column: str


In [9]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories



class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 schema_filepath= SCHEMA_FILE_PATH,
                 param_filepath = PARAM_FILE_PATH):
        self.config=read_yaml(CONFIG_FILE_PATH)
        self.params=read_yaml(PARAM_FILE_PATH)
        self.schema=read_yaml(SCHEMA_FILE_PATH)

        print(self.schema.TARGET_COLUMNS)
        print(self.schema.TARGET_COLUMNS.name)


        create_directories([self.config.artifacts_roots])
        
    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config=self.config.model_trainer
        schema=self.schema.TARGET_COLUMNS

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            target_column=schema.name
        )

        return model_trainer_config

In [10]:
from mlProject import logger
from sklearn.linear_model import LinearRegression
import joblib
import pandas as pd


class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config=config

    def train(self):
        train_data=pd.read_csv(self.config.test_data_path)
        test_data=pd.read_csv(self.config.test_data_path)

        train_x=train_data.iloc[:,:-1]
        test_x=test_data.iloc[:,:-1]
        train_y=train_data.iloc[:, [-1]]
        test_y=test_data.iloc[:, [-1]]

        lr=LinearRegression()
        lr.fit(train_x,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
            





In [11]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer_config= ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e 


[2025-11-11 22:57:04,629:INFO:common:yaml file <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='utf-8'> loaded succesfully]
[2025-11-11 22:57:04,630:INFO:common:yaml file <_io.TextIOWrapper name='params.yaml' mode='r' encoding='utf-8'> loaded succesfully]
[2025-11-11 22:57:04,634:INFO:common:yaml file <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='utf-8'> loaded succesfully]
{'name': 'performance_index'}
performance_index
[2025-11-11 22:57:04,635:INFO:common:created directories artifacts]
[2025-11-11 22:57:04,636:INFO:common:created directories artifacts/model_trainer]


NameError: name 'self' is not defined

In [26]:
import pandas
pd.read_csv("artifacts/data_transformation/test_model_data_csv")

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.277920,-0.500199,-0.212807,0.862116,-1.735221,1.362366,-0.435485,-0.256571,-0.035141,67.858582
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.536699,0.740907,0.911499,-1.231149,0.533740,-0.833199,0.572503,0.671948,0.939923,224.426784
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-1.760472,-0.165578,-0.925500,-0.791620,-0.528494,-0.263903,1.359736,-0.118091,-0.160475,80.496807
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.841307,-0.205618,0.155768,-1.180397,0.346562,-0.833199,-1.402294,-0.283044,0.320206,111.396486
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.032669,0.109281,0.504218,-1.577792,1.635987,1.247075,-0.373725,-0.046932,-0.085546,155.027950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-0.502469,-0.188132,-0.404106,-0.038007,0.235360,0.203701,0.291703,-0.212955,-0.179314,94.308886
121,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.556617,-0.401636,-0.521629,-1.032930,-0.590101,-0.833199,-0.699869,-0.209570,-0.283487,70.313053
122,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.665130,-0.016837,0.165039,0.429291,1.620527,0.242486,-1.879588,-0.275154,-0.381113,128.381051
123,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,-0.445117,-0.451532,-0.496650,-1.134433,-1.092399,-0.377039,-0.057619,-0.216484,-0.206608,66.162600
